# Parkinson's Disease Progression Prediction
### Project - SI 618: Data Manipulation and Analysis

Authors
1. Nowrin Mohamed - nowrin@umich.edu
2. Prithvijit Dasgupta - prithvid@umich.edu
3. Sachin Salim - sachinks@umich.edu

In [1]:
'''
Kaggle Competition:
@misc{amp-parkinsons-disease-progression-prediction,
    author = {Leslie Kirsch, Sohier Dane, Stacey Adam, Victoria Dardov},
    title = {AMP®-Parkinson's Disease Progression Prediction},
    publisher = {Kaggle},
    year = {2023},
    url = {https://kaggle.com/competitions/amp-parkinsons-disease-progression-prediction}
}
''';

References:
1. Baseline model - https://www.kaggle.com/code/renataghisloti/linearregression-simple-57-3-smape

In [38]:
import os

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
supplemental_clinical_data = pd.read_csv('data/supplemental_clinical_data.csv')
peptides_data = pd.read_csv('data/train_peptides.csv') 
clinical_data = pd.read_csv('data/train_clinical_data.csv') 
proteins_data = pd.read_csv('data/train_proteins.csv')

In [43]:
def read_processed_data(filename):
    df_loaded = pd.read_csv(f'processed_data/{filename}')
    df_loaded.set_index(['visit_id', 'patient_id', 'visit_month'], inplace=True)
    return df_loaded

## Feature extraction

### Handling missing data

In [44]:
def fill_na(data):
    data_1 = data.groupby('patient_id').transform(lambda x: x.fillna(x.mean()))
    data_2 = data_1.groupby('visit_month').transform(lambda x: x.fillna(x.mean()))
    data_3 = data_2.fillna(data_2.mean())
    return data_3

### Load processed features

In [45]:
def load_proteins_features():
    filename = 'proteins_features.csv'
    if os.path.isfile(f'processed_data/{filename}'):
        proteins_features = read_processed_data(filename)
    else:
        proteins_features = proteins_data.pivot(index=['visit_id', 'patient_id', 'visit_month'], columns='UniProt', values='NPX')

        # proteins_nan_mask = proteins_features.isna()
        proteins_features.head()

        proteins_features = fill_na(proteins_features)

        proteins_features.to_csv(f'processed_data/{filename}')

    return proteins_features


In [47]:
proteins_features = load_proteins_features()
proteins_features.head()

,,,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,patient_id,visit_month,,,,,,,,,,,,,,,,,,,,,
10053_0,10053,0,9104.27,402321.0,7126.96,24525.7,7150.57,2497.840,83002.9,15113.6,167327.0,129048.0,...,317477.0,9469.45,94237.6,14907.159185,23016.0,177983.0,65900.0,15382.00,9295.65,19017.40
10053_12,10053,12,10464.20,435586.0,7126.96,24525.7,7150.57,2435.275,197117.0,15099.1,164268.0,108114.0,...,317477.0,14408.40,102672.3,14659.712042,28537.0,171733.0,65668.1,13097.65,9295.65,25697.80
10053_18,10053,18,13235.70,507386.0,7126.96,24525.7,7150.57,2372.710,126506.0,16289.6,168107.0,163776.0,...,317477.0,38667.20,111107.0,17048.833333,37932.6,245188.0,59986.1,10813.30,9295.65,29102.70
10138_12,10138,12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.900,156313.0,54546.4,204013.0,56725.0,...,557904.0,44556.90,155619.0,14647.900000,36927.7,229232.0,106564.0,26077.70,21441.80,7642.42
10138_24,10138,24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.150,151169.0,52338.1,240892.0,85767.1,...,495006.0,47836.70,177619.0,17061.100000,25510.4,176722.0,59471.4,12639.20,15091.40,6168.55


In [48]:
def load_peptides_features():
    filename = 'peptides_features.csv'
    if os.path.isfile(f'processed_data/{filename}'):
        peptides_features = read_processed_data(filename)
    else:
        peptides_features = peptides_data.pivot(index=['visit_id', 'patient_id', 'visit_month'], columns='Peptide', values='PeptideAbundance')

        # peptides_nan_mask = peptides_features.isna()
        peptides_features.head()

        peptides_features = fill_na(peptides_features)

        peptides_features.to_csv(f'processed_data/{filename}')

    return peptides_features


In [49]:
peptides_features = load_peptides_features()
peptides_features.head()

,,Peptide,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,ADDKETC(UniMod_4)FAEEGK,ADDKETC(UniMod_4)FAEEGKK,ADDLGKGGNEESTKTGNAGSR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,patient_id,visit_month,,,,,,,,,,,,,,,,,,,,,
10053_0,10053,0,6580710.0,31204.4,7735070.0,35984.7,17188.0,19787.3,46620.3,236144.0,2556837.7,30838.2,...,202274.0,8926.159054,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,50835.5,7207.30
10053_12,10053,12,6333510.0,52277.6,5394390.0,35984.7,17188.0,19787.3,57554.5,108298.0,45885.4,30838.2,...,201009.0,8682.690196,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,10053,18,7129640.0,61522.0,7011920.0,35984.7,17188.0,19787.3,36029.4,708729.0,5067790.0,30838.2,...,220728.0,10192.130714,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,10138,12,7404780.0,46107.2,10610900.0,20093.2,20910.2,66662.3,55253.9,79575.5,6201210.0,26720.0,...,188362.0,9433.710000,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,10138,24,13788300.0,56910.3,6906160.0,13785.5,11004.2,63672.7,36819.8,34160.9,2117430.0,15645.2,...,206187.0,6365.150000,3521800.0,69984.6,496737.0,80919.3,111799.0,568923.5,56977.6,4903.09


In [29]:
clinical_features = clinical_data.drop(
    columns=["upd23b_clinical_state_on_medication"]).copy()
clinical_features.set_index(['visit_id', 'patient_id', 'visit_month'], inplace=True)

# nan mask
clinical_nan_mask = clinical_features.isna()

clinical_features = fill_na(clinical_features)
clinical_features = clinical_features.astype({'updrs_1': 'int64', 'updrs_2': 'int64', 'updrs_3': 'int64', 'updrs_4': 'int64'})
print(clinical_features.shape)
clinical_features.head()

(2615, 4)


,,,updrs_1,updrs_2,updrs_3,updrs_4
visit_id,patient_id,visit_month,,,,
55_0,55,0,10,6,6,0
55_3,55,3,10,7,19,0
55_6,55,6,8,10,58,0
55_9,55,9,8,9,17,0
55_12,55,12,10,10,2,0


In [ ]:
common_indices = np.intersect1d(peptide_features.index.values,
     clinical_features.index.values)
print(f"{len(common_indices)} visits are common between input\
 and output features")

In [ ]:
X_data = proteins_features
y_data = clinical_features

X_data = X_data.loc[common_indices]
y_data = y_data.loc[common_indices]

print(X_data.shape)
print(y_data.shape)

In [ ]:
X_data.head()

In [ ]:
y_data.head(2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# TODO: split based on patients (?)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_test_ = y_test.copy()
y_test_.columns = "t_" + y_test.columns.astype(str)
# pd.MultiIndex.from_product(
#         [['true'], y_test.columns],
#         names=['', 'updrs'])
        
y_pred_ = pd.DataFrame(y_pred)
y_pred_.index = y_test_.index
y_pred_.columns = "p_" + y_test.columns.astype(str)
# pd.MultiIndex.from_product(
#         [['pred'], y_test.columns],
#         names=['', 'updrs'])

result_df = pd.concat([y_test_, y_pred_], axis=1)
result_df.shape

In [ ]:
result_df.head()

In [ ]:
clinical_nan_mask.head()

In [ ]:
def visit_to_patmonth(df, reverse = False):
    '''converts visits to patient-month'''
    # TODO: replace clinical data with whole data
    if not reverse:
        df.index = pd.MultiIndex.from_frame(
            clinical_data.query('visit_id in @df.index')[['patient_id', 'visit_month']])
    else:
        df.index = pd.Index(df.index.to_series().apply(lambda x: str(x[0])+"_"+str(x[1])), name='visit_id')
    return df

In [ ]:
# result_df = visit_to_patmonth(result_df, False)

In [ ]:
# result_df.groupby('patient_id').count()['true']['updrs_1'].sort_values(ascending = False)

In [ ]:
result_df.head(2)

In [ ]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true != 0)|(y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [ ]:
result_df[['t_updrs_1', 'p_updrs_1']]
plt.scatter(result_df['t_updrs_1'], result_df['p_updrs_1'])

In [ ]:
smape(result_df['t_updrs_1'],
    result_df['p_updrs_1'])

In [ ]:
print(clinical_nan_mask.shape)
print(result_df.shape)

# Clinical data

`supplemental_clinical_data` contains patients whose proteins were never measured.

`clinical_data` patients had their proteins measured, but not on all visits

### y extraction

TODO: Use `upd23b_clinical_state_on_medication` as X feature

In [ ]:
clinical_features = clinical_data[['visit_id', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
clinical_features.set_index('visit_id', inplace=True)

In [ ]:
X_indices = peptide_features.index
y_indices = clinical_features.index

common_indices = np.intersect1d(X_indices.values, y_indices.values)
print(f"{len(common_indices)} items are common between input features\
 and clinical data")

miss_indices = np.setdiff1d(X_indices.values, y_indices.values)
print(f"{len(miss_indices)} items in input features are missing in clinical data")

Transforming clinical data

In [ ]:
def add_month_columns(df, add_month):
    multi_cols = pd.MultiIndex.from_product(
        [[add_month], df.columns],
        names=['add_month', 'updrs'])

    # Add the MultiIndex to the DataFrame
    df.columns = multi_cols
    return df

def df_with_n_extra_month(clinical_df, add_month = 0):
    clinical_extra_df = clinical_df.copy()
    # setting to nan as default
    clinical_extra_df.loc[:] = np.nan

    clinical_extra_df.index = clinical_extra_df.index.set_levels(
        clinical_extra_df.index.get_level_values('visit_month') + add_month,
        level=1)
    clinical_extra_df

    clinical_df = clinical_df.droplevel(0)
    clinical_extra_df = clinical_extra_df.droplevel(0)

    common_indices = pd.Index(np.intersect1d(clinical_df.index.values, clinical_extra_df.index.values))

    clinical_extra_df.loc[
        common_indices,
        clinical_df.columns] = clinical_df.loc[
            common_indices,
            clinical_df.columns].values

    return clinical_extra_df
    
def transform_clinical_data(clinical_df):
    # select indices
    clinical_df.set_index(['patient_id', 'visit_month'], inplace=True)

    all_clinical_extras = []
    for extra_month in [0, 6, 12, 24]:
        clinical_extra = df_with_n_extra_month(clinical_df, add_month = extra_month)
        clinical_extra = add_month_columns(clinical_extra, add_month = extra_month)
        clinical_extra.reset_index(drop=True, inplace=True)
        all_clinical_extras.append(clinical_extra)
        
    clinical_features_final = pd.concat(all_clinical_extras, axis=1) #.set_index('visit_month')
    clinical_features_final.index = clinical_df.index

    return clinical_features_final

In [ ]:
clinical_data.columns

In [ ]:
clinical_features = clinical_data.drop(columns="upd23b_clinical_state_on_medication").copy()
clinical_features = clinical_features.query('patient_id == 55')

In [ ]:
clinical_features = clinical_data.drop(
    columns=["visit_id", "upd23b_clinical_state_on_medication"]).copy()
clinical_features = clinical_features.query('patient_id == 55')

clinical_features_final = transform_clinical_data(clinical_features)
print(clinical_features_final.shape)
clinical_features_final

### Filling nan

Filling with average of each updrs

In [ ]:
mean_values = clinical_features.mean(numeric_only=True).astype(int)
clinical_features.fillna(mean_values, inplace=True)
clinical_features = clinical_features.astype({'updrs_1': 'int64', 'updrs_2': 'int64', 'updrs_3': 'int64', 'updrs_4': 'int64'})
display(clinical_features.head(7))
display(clinical_features.dtypes)

In [ ]:
clinical_features['updrs_3'].plot.hist()